In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from datetime import date,timedelta
import time

In [13]:
url = f"https://www.worldometers.info/coronavirus/"
browser = webdriver.Chrome(executable_path="Driver/chromedriver.exe")
browser.get(url)
browser.find_element_by_link_text('Yesterday').click()

In [15]:
html_text = browser.page_source
tree = BeautifulSoup(html_text, 'html.parser')
tbody = tree.find("tbody")
trows = tbody.find_all("tr",{"role": "row"})
total_row_world = tbody.find_all("tr",{"class" : "total_row_world"})
list_data_country = [trow.text.split("\n")[1:-2] for trow in trows if trow not in total_row_world]
list_data_continent = []
for trow in total_row_world:
    if "World" not in trow.text:
        temp_data = trow.text.split("\n")[3:-7]
        temp_data[1] = temp_data[0]
        list_data_continent.append(temp_data[1:])

In [16]:
list_data_country

[['1',
  'USA',
  '32,308,557',
  '+2,645',
  '579,951 ',
  '+9',
  '24,836,187',
  '6,892,419',
  '9,712',
  '97,158',
  '1,744',
  '426,562,606',
  '1,282,751',
  '332,537,343 '],
 ['2',
  'India',
  '14,526,609',
  '+4,926',
  '175,673 ',
  '',
  '12,671,220',
  '1,679,716',
  '8,944',
  '10,445',
  '126',
  '264,972,022',
  '190,529',
  '1,390,716,194 '],
 ['3',
  'Brazil',
  '13,834,342',
  '',
  '369,024 ',
  '',
  '12,298,863',
  '1,166,455',
  '8,318',
  '64,721',
  '1,726',
  '28,600,000',
  '133,799',
  '213,753,558 '],
 ['4',
  'France',
  '5,224,321',
  '',
  '100,404 ',
  '',
  '4,046,205',
  '1,077,712',
  '5,914',
  '79,897',
  '1,536',
  '71,424,719',
  '1,092,324',
  '65,387,859 '],
 ['5',
  'Russia',
  '4,693,469',
  '+9,321',
  '105,193 ',
  '+398',
  '4,319,389',
  '268,887',
  '2,300',
  '32,151',
  '721',
  '125,100,000',
  '856,943',
  '145,984,033 '],
 ['6',
  'UK',
  '4,383,572',
  '',
  '127,225 ',
  '',
  '4,139,553',
  '116,794',
  '329',
  '64,305',
  '1,86

In [4]:
df_continent = pd.DataFrame(list_data_continent[:-1], columns = ["Continent", "Total Cases", "New Cases", "Total Deaths", 
                                        "New Death", "Total Recovered", "Active Case", "Serious Critical"])
df_continent = df_continent.replace([""," "], "0")
df_continent["Time"] = date.today() - timedelta(1)
df_continent.head()

,Continent,Total Cases,New Cases,Total Deaths,New Death,Total Recovered,Active Case,Serious Critical,Time
0,Europe,"42,365,606","+199,467","966,569","+3,864","36,604,467","4,794,570","32,929",2021-04-15
1,North America,"37,184,476","+95,739","842,974","+1,568","28,860,618","7,480,884","16,873",2021-04-15
2,Asia,"32,588,924","+371,874","458,542","+2,726","28,639,217","3,491,165","28,756",2021-04-15
3,South America,"23,031,584","+156,256","612,214","+5,357","20,649,343","1,770,027","24,332",2021-04-15
4,Africa,"4,433,974","+12,678","117,363",+313,"3,963,106","353,505","3,812",2021-04-15


In [5]:
df_country = pd.DataFrame(list_data_country, columns = ["#", "Country, Other", "Total Cases", "New Cases", "Total Deaths", 
                                        "New Death", "Total Recovered", "Active Case", "Serious Critical", 
                                        "Tot Cases/1M pop", "Death/1M pop","Total test","Test/1M pop", "Population"])
df_country.set_index("#", inplace=True)
df_country["Time"] = date.today() - timedelta(1)
df_country = df_country.replace([""," "], "0")
df_country.head()

,"Country, Other",Total Cases,New Cases,Total Deaths,New Death,Total Recovered,Active Case,Serious Critical,Tot Cases/1M pop,Death/1M pop,Total test,Test/1M pop,Population,Time
#,,,,,,,,,,,,,,
1,USA,"32,224,080","+74,420","578,993",+895,"24,770,658","6,874,429","9,762","96,905","1,741","424,867,753","1,277,675","332,532,050",2021-04-15
2,India,"14,287,740","+216,850","174,335","+1,183","12,543,978","1,569,427","8,944","10,274",125,"262,003,415","188,400","1,390,679,153",2021-04-15
3,Brazil,"13,758,093","+80,529","365,954","+3,774","12,236,295","1,155,844","8,318","64,366","1,712","28,600,000","133,802","213,749,436",2021-04-15
4,France,"5,187,879","+38,045","100,073",+296,"4,008,331","1,079,475","5,924","79,341","1,530","71,017,438","1,086,102","65,387,466",2021-04-15
5,Russia,"4,675,153","+8,944","104,398",+398,"4,301,448","269,307","2,300","32,025",715,"124,700,000","854,204","145,983,862",2021-04-15


In [6]:
writer = pd.ExcelWriter("data.xlsx", engine='xlsxwriter',datetime_format = "YYYY-MM-DD")
# df_continent.to_excel(writer, sheet_name= "continent")
# df_country.to_excel(writer, sheet_name= "country")
# writer.save()

In [7]:
old_df_continent = pd.read_excel(writer, sheet_name= "continent", index_col=0)
frames1 = [old_df_continent, df_continent]
result1 = pd.concat(frames1)
result1.to_excel(writer, sheet_name= "continent")
#
old_df_country = pd.read_excel(writer, sheet_name= "country", index_col=0)
frames2 = [old_df_country, df_country]
result2 = pd.concat(frames2)
result2.to_excel(writer, sheet_name= "country")
writer.save()